In [1]:
import ui.components as uic
import pandas as pd

from ui.screens import add_entries
from IPython.display import display
from rich import print as rprint

from api.internal import TachiBk, internDB
from notes.notes import notesDB



notes = notesDB()
tachidb = TachiBk()
interndb = internDB()

from_url   = {'notes': []}
from_libby = {'notes' : []}
from_tachi = {'notes': []}
search = {'notes': [], 'title': None}
manual = {'notes': [], 'entries': [], 'lastID': interndb.getLastID()+1}

In [ ]:
'''Screen switching note entry's external DB.

Parameter: notesDB object.
'''

entry_id = uic.entry_id()
entry_id.layout = {'width': "90%"}

api_dropdown = uic.widgets.Dropdown(options = uic.SEARCH_API | uic.URL_API, layout = {'width': '200px'},
                                    style = {'background':'transparent', 'text_color': 'var(--vscode-editor-foreground)'})

dropdowns = uic.dropdowns(horizontal=False)
note_input = uic.notes_input(description="Notes:", width="90%", height="300px")
currentDB = uic.widgets.HTML(value = "<b>CURRENT DB:</b> ")
currentDB_output = uic.show_notes_entry(notes.getByID(entry_id.value), notes, 
                                        dropdowns, note_input, currentDB)
newDB_output = uic.widgets.HTML(layout={'width': '90%', 'height': '350px'})
entryHeader = uic.widgets.HTML(value=f"<h2>{notes.getByID(entry_id.value)['title']} by {notes.getByID(entry_id.value)['authors']}  </h2>")
entry_selection = uic.widgets.Select(layout = {'width': '300px', 'height': '150px'},
                                        style = {'background':'transparent', 'text_color': 'var(--vscode-editor-foreground)'} )
url = uic.widgets.Text(placeholder="Enter supported URL (IMDB, Goodreads) ...",
                    layout = {'width':'100%','align': 'right', 'display':'None'})

##########################

def on_id_change(change):
    entry = notes.getByID(entry_id.value)
    oldAPI = notes.API[entry['DB'].split("/")[0]]
    extern = notes.get_external_entry(entry)

    note_input.value = entry['notes']
    entryHeader.value = f"<h2>{entry['title']} by {entry['authors']}</h2>"
    dropdowns.children[0].value = entry['progress']
    dropdowns.children[1].value = entry['type']
    currentDB.value = f"<b>CURRENT DB:</b> {entry['DB'].split("/")[0].upper()}"
    currentDB_output.value= oldAPI.to_html(extern)
    uic.api_search_2(api_dropdown, entry['title'], entry_selection, url, newDB_output)

def on_notes_change(change):
    notes.df.at[entry_id.value, 'notes'] = note_input.value

def on_progress_change(change):
    notes.df.at[entry_id.value, 'progress'] = dropdowns.children[0].value

def on_type_change(change):
    notes.df.at[entry_id.value, 'type'] = dropdowns.children[1].value

def on_selection_change(change):
    api = api_dropdown.value
    if entry_selection.value != "NO RESULTS FOUND" and api in uic.SEARCH_API.values():
        selection = api.getByID(api.getEntryID(entry_selection.value))
        newDB_output.value = api.to_html(selection)
    else:
        newDB_output.value = "NO RESULTS FOUND"

def on_url_change(change):
    api = api_dropdown.value
    if api in uic.URL_API.values():
        selection = api.getEntryFromURL(url.value)
        if 'warning' in selection.keys():
            newDB_output.value = selection['warning']
        else:
            newDB_output.value = api.to_html(selection)

def on_api_change(change):
    entry = notes.getByID(entry_id.value)
    api = uic.api_search_2(api_dropdown, entry['title'], entry_selection, url, newDB_output)
    if entry_selection.value != "NO RESULTS FOUND":
        selection = api.getByID(api.getEntryID(entry_selection.value))
        newDB_output.value = api.to_html(selection)
    else:
        newDB_output.value = "NO RESULTS FOUND"

url.observe(on_url_change, names='value')
entry_id.observe(on_id_change, names='value')
note_input.observe(on_notes_change, names='value')
dropdowns.children[0].observe(on_progress_change, names='value')
dropdowns.children[1].observe(on_type_change, names='value')
entry_selection.observe(on_selection_change, names='value')
api_dropdown.observe(on_api_change, names='value')

##########################
def process(b):
    new_api = api_dropdown.value
    if new_api in uic.SEARCH_API.values():
        if entry_selection.value != "NO RESULTS FOUND":
            new_id = new_api.getEntryID(entry_selection.value)
            notes.df.at[entry_id.value, 'DB'] = f"{new_api.name}/{new_id}"
    elif new_api in uic.URL_API.values():
        selection = new_api.getEntryFromURL(url.value)
        new_id = new_api.getEntryID(selection)
        notes.df.at[entry_id.value, 'DB'] = f"{new_api.name}/{new_id}"
        
    # refresh note entry output
    entry = notes.getByID(entry_id.value)
    oldAPI = notes.API[entry['DB'].split("/")[0]]
    extern = notes.get_external_entry(entry)
    currentDB.value = f"<b>CURRENT DB:</b> {entry['DB'].split("/")[0].upper()}"
    currentDB_output.value= oldAPI.to_html(extern)
        
accept_button = uic.general_accept_button(process, "Switch DB")
accept_button.layout = {'align': 'left'}
##########################

note_output = uic.widgets.VBox([entry_id, dropdowns, note_input, currentDB])
DB = uic.widgets.Accordion([currentDB_output], titles=("Entry",""), layout={'width':'75%'}, selected_index=0)
api_menu = uic.widgets.HBox([api_dropdown, accept_button])
switch = uic.widgets.VBox([api_menu, url, entry_selection, newDB_output, ])
switchCollapse = uic.widgets.Accordion([switch], titles=('Switch DB',""))
left = uic.widgets.VBox([note_output, switchCollapse])
uic.widgets.HBox([left, DB])